In [1]:
# Import packages
import numpy as np
import condynsate

In [2]:
# Create an instance of the simulator with visualization and animation
sim = condynsate.Simulator(visualization=True,
                           animation=True,
                           animation_fr=15.)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [3]:
# Load the station
station_obj = sim.load_urdf(urdf_path='./segbot_vis/station.urdf',
                            position=[0., 0., 19.355],
                            roll=np.pi/2.0,
                            fixed=True,
                            update_vis=True)

In [4]:
# Load the segbot
segbot_obj = sim.load_urdf(urdf_path='./segbot_vis/segbot.urdf',
                           position=[0., 0., 0.],
                           yaw=np.pi,
                           fixed=False,
                           update_vis=True)

In [5]:
# Set the chassis and wheel masses
sim.set_link_mass(urdf_obj=segbot_obj,
                  link_name="chassis",
                  mass=20.)
sim.set_link_mass(urdf_obj=segbot_obj,
                  link_name="right_wheel",
                  mass=20.)
sim.set_link_mass(urdf_obj=segbot_obj,
                  link_name="left_wheel",
                  mass=20.)

In [6]:
# Variables to track applied torque
max_torque = 10.
min_torque = -10.

# Keyboard IO function for torque
def get_torque_setting(min_torque, max_torque):
    if sim.is_pressed("shift+w"):
        r_torque = 0.75 * max_torque
        l_torque = 0.75 * max_torque
    elif sim.is_pressed("w"):
        r_torque = 0.33 * max_torque
        l_torque = 0.33 * max_torque
    elif sim.is_pressed("shift+s"):
        r_torque = 0.75 * min_torque
        l_torque = 0.75 * min_torque
    elif sim.is_pressed("s"):
        r_torque = 0.33 * min_torque
        l_torque = 0.33 * min_torque
    else:
        r_torque = 0.0
        l_torque = 0.0
    if sim.is_pressed("d"):
        l_torque = l_torque + 0.25*max_torque
    if sim.is_pressed("a"):
        r_torque = r_torque + 0.25*max_torque
    return r_torque, l_torque

In [7]:
# Variables to track station velocity
max_stat_vel = 0.05
min_stat_vel = 0.
stat_vel = 0.5 * (min_stat_vel + max_stat_vel)

# Keyboard IO data for station vel
def get_stat_vel_setting(stat_vel, min_stat_vel, max_stat_vel):
    if sim.is_pressed("e"):
        stat_vel = stat_vel + 0.005*(max_stat_vel - min_stat_vel)
        if stat_vel > max_stat_vel:
            stat_vel = max_stat_vel
    elif sim.is_pressed("q"):
        stat_vel = stat_vel - 0.005*(max_stat_vel - min_stat_vel)
        if stat_vel < min_stat_vel:
            stat_vel = min_stat_vel
    return stat_vel

In [8]:
# Create desired plots
plot_1, lines_1 = sim.add_subplot(n_lines=1,
                                  title="Torque vs Time",
                                  x_label="Time [s]",
                                  y_label="Torque [Nm]",
                                  colors=["r"])
plot_2, lines_2 = sim.add_subplot(n_lines=1,
                                  title="Pitch Speed Vs Time",
                                  x_label="Time [s]",
                                  y_label="Pitch Velocity [Rad/s]",
                                  colors=["b"])

In [9]:
# Open the animator GUI
sim.open_animator_gui()

In [10]:
# Main simulation loop
while not sim.is_done: 
    # Get the torques and station vel
    r_torque, l_torque = get_torque_setting(min_torque, max_torque)
    stat_vel = get_stat_vel_setting(stat_vel, min_stat_vel, max_stat_vel)

    # Set wheel torques
    sim.set_joint_torque(urdf_obj=segbot_obj,
                         joint_name='chassis_to_right_wheel',
                         torque=r_torque,
                         show_arrow=True,
                         arrow_scale=0.075,
                         arrow_offset=-0.05)
    sim.set_joint_torque(urdf_obj=segbot_obj,
                         joint_name='chassis_to_left_wheel',
                         torque=l_torque,
                         show_arrow=True,
                         arrow_scale=0.075,
                         arrow_offset=0.05)

    # Set station velocity
    sim.set_joint_velocity(urdf_obj=station_obj,
                           joint_name="world_to_station",
                           velocity=stat_vel,
                           color=True,
                           min_vel=min_stat_vel,
                           max_vel=max_stat_vel)

    # Get the rigid body states of the segbot and station
    seg_state = sim.get_base_state(urdf_obj=segbot_obj,
                                   body_coords=True)
    stat_state = sim.get_base_state(urdf_obj=station_obj,
                                    body_coords=False)
    seg_pos = seg_state['position']
    seg_ang_vel = seg_state['angular velocity']
    stat_pos = stat_state['position']

    # This simulates centrifugal gravity from the station
    seg_relative_pos = np.array(seg_pos) - np.array(stat_pos)
    seg_relative_dirn = seg_relative_pos / np.linalg.norm(seg_relative_pos)
    gravity = 9.81 * seg_relative_dirn
    sim.set_gravity(gravity=gravity)

    # Set the plot data
    sim.add_subplot_point(subplot_index=plot_1,
                          line_index=lines_1[0],
                          x=sim.time,
                          y=0.5*(r_torque + l_torque))
    sim.add_subplot_point(subplot_index=plot_2,
                          line_index=lines_2[0],
                          x=sim.time,
                          y=seg_ang_vel[1])

    # Step the sim
    sim.step(real_time=True,
             update_vis=True,
             update_ani=True)
    

RESETTING...
RESETTING...
RESETTING...
Termination command detected. Terminating keyboard listener. Goodbye
